In [234]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [235]:
# Constants

nP = 73 # number of people
nT = 210 # number of timeslots in a week
# Organized in 30 minute interals from 9am-12pm, Sunday-Saturday
# 30 intervals/day * 7 days/week
cM = 1e7# multiplier for the minimal matching objective

print("")

In [236]:
# Load Data

# 0-1 matrix of whether person can mentor/mentee at time T
load_A = Matrix(CSV.read("schedules.csv", DataFrame, header=0))[3:end, 2:end]  # nP x nT
A = zeros(nP, nT)
for i in 1:nP, j in 1:nT
    A[i, j] = parse(Int64, load_A[i, j])
end

@show size(A)

# for these, what if someone is only a mentor or only a mentee? would it be zero?
aR = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 2] # nP x 1: max num of session person can mentor
aE = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 3] # nP x 1: max num of sessions person can mentee (0 or 1)
# 
# aE = Matrix(CSV.read("max_menteeing.csv", DataFrame)) # nP x 1: max num of session person can mentee (can we remove this?)
r  = Matrix(CSV.read("rank.csv", DataFrame, header=0))[:, 2]# nP x 1, rank

# Set ians rank down so he gets a mentor...
r[30] = -2

# Don't we just need one per person? Lessens the amount of data, plus not everyone is a mentor or mentee
p = Matrix(CSV.read("poomsae_pref.csv", DataFrame, header=0))[:, 2]
s = Matrix(CSV.read("sparring_pref.csv", DataFrame, header=0))[:, 2]
# pR = Matrix(CSV.read("mentor_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer poomsae 
# pE = Matrix(CSV.read("mentee_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentee prefer poomsae
# sR = Matrix(CSV.read("mentor_sparring.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer sparring
# sE = Matrix(CSV.read("mentee_sparring.csv")) # nP x 1: 0-1 does mentee prefer sparring

#@show aR
@show aE
#@show r
#@show p
#@show s


print("")

size(A) = (73, 210)
aE = Any[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [237]:
# Create Cost Matrix

alpha = 1
beta  = 10
gamma = 100


# Break up the objectives for different analysis
qA = zeros(nP, nP)  # quality of match: availability
qR = zeros(nP, nP)  # quality of match: rank
qI = zeros(nP, nP)  # quality of match: interest

# Adjusting to only have one set of data for sparring and poomsae preferences
for i = 1:nP, j = 1:nP
    # @show(A[i, :]'A[j, :])  # why is this zero or 1
    #print(A[j])
    qA[i, j] += (A[i, :]'A[j, :])
    qR[i, j] += (r[j]-r[i]+2-(r[j] <= 0))
    qI[i, j] += (p[i]*p[j]+s[i]*s[j]-p[i]*s[i]-p[j]*s[j])
end

C = -alpha .* qA + beta .* qR - gamma .* qI .- 1


qI

# for i = 1:nP, j = 1:nP
#     C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
#     C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
#     C[i, j] -= 1
# end


73×73 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     1.0  1.0  1.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [238]:
# Rank Constraint Matrix
# Adjust to be negative hehe

R = zeros(nP, nP)

for i = 1:nP, j = 1:nP
    if r[i] <= r[j]-2 || (r[i] <= r[j]-1 && r[j] <= 0)
        R[i, j] = 1
    end
end

R = convert(Array{Int64}, R);

In [239]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_optimizer_attribute(m, "IntegralityFocus",1)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03
Set parameter IntegralityFocus to value 1


In [240]:
@variable(m, 0 <= x[1:nP, 1:nP] <= 1)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [241]:
# Rank Constraint (capacity constraint)
@constraint(m, x .<= R);

In [242]:
# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P] for i=1:nP) <= aE[P]);
    @constraint(m, sum(x[P, i] for i=1:nP) <= aR[P]); # replaced with ones
end

In [243]:
# # Availability Constraints
# for i = 1:nP, j = i+1:nP, T=1:nT
#     @constraint(m, x[i,j,T] <= A[i,T]*A[j,T]);
# end

In [244]:
# # THIS ONE TAKES TOO LONG

# # Single Mentor Constraint
# for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
#     @constraint(m, x[i,k,T]+x[j,k,T] <= 1);
# end

In [245]:
# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P] for i=1:nP) >= t[P])
    @constraint(m, 0.01*sum(x[i,P] for i=1:nP) <= t[P]);  # make constraint integral
end

print("")

In [246]:
@objective(m, Min, cM*sum((1-t[P])*aE[P] for P=1:nP) + sum(x[i,j]*C[i,j] for i=1:nP, j=1:nP))
print("")

In [247]:
optimize!(m)

objective_value(m)

Set parameter IntegralityFocus to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-6950X CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5621 rows, 5402 columns and 26791 nonzeros
Model fingerprint: 0xe567f90a
Variable types: 5329 continuous, 73 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 6.200000e+08
Presolve removed 5532 rows and 4013 columns
Presolve time: 0.02s
Presolved: 89 rows, 1389 columns, 2716 nonzeros
Variable types: 1327 continuous, 62 integer (62 binary)
Found heuristic solution: objective 1.000083e+07

Root relaxation: objective -1.324000e+03, 206 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |  

-1324.0

In [248]:
count = 0
x_sol = value.(x)
for i in 1:nP, j in 1:nP
    if x_sol[i, j] != 0
        count += 1
    end
end
@show count

count = 62


62

In [249]:
names = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 1]
# @show names

# count = 0
# x_sol = value.(x)
for i in 1:nP, j in 1:nP
    if x_sol[i, j] != 0
        print("$(names[i]), $(names[j]), $(x_sol[i, j])\n")
    end
end
# @show count

Akila Saravanan, Claire Kim, 1.0
Ana Uvarova, Anas Chentouf, 1.0
Ana Uvarova, Ian Rosado, 1.0
Andy Lambert, Bianca Santi, 1.0
Andy Lambert, Tiffany Louie, 1.0
Angela Gao, Crystal Liang, 1.0
Angela Gao, Evelyn Fu, 1.0
Bianca Santi, Esther Kinyanjui, 1.0
Bianca Santi, Fatima Zaidouni, 1.0
Bint Outhman, Arthur Liang, 1.0
Bint Outhman, Brian Le, 1.0
Chanwoo Yoon, Diego Coello, 1.0
Chanwoo Yoon, Titus Tsai, 1.0
Cher Jiang, Ophelia Zhu, 1.0
Cher Jiang, Pierce Lai, 1.0
Christine Tu, Cameron Kokesh, 1.0
Christine Tu, Jonathan Zong, 1.0
Claire Kim, Ana Uvarova, 1.0
Claire Kim, Angela Gao, 1.0
Claire Kim, Bint Outhman, 1.0
Emmie Le Roy, Enid M Cruz Colón, 1.0
Emmie Le Roy, Nurullah Giray Kuru, 1.0
Evelyn Fu, Isabela Fuentes, 1.0
Evelyn Fu, Pravalika Putalapattu, 1.0
Grace Huang, Anna Rasmussen, 1.0
Grace Huang, Karissa Sanchez, 1.0
Grace Jau, Robert van der Drift, 1.0
Ian Williams, Alonso Hernandez, 1.0
Ian Williams, Xavier Sanchez, 1.0
Jawad Yousef, Austin White, 1.0
Jawad Yousef, Isa Liggans, 

In [250]:
```
Analysis:
-Generate an x representing the existing mentorship pairs and calculate the cost
-Report how many people who wanted a mentor received one
-

```

`Analysis: -Generate an x representing the existing mentorship pairs and calculate the cost -Report how many people who wanted a mentor received one -`

In [251]:
baseline_x_load = Matrix(CSV.read("baseline.csv", DataFrame, header=0))[2:end, 2:end]
baseline_x = zeros(nP, nP)
for i in 1:nP, j in 1:nP
    baseline_x[i, j] = parse(Int64, baseline_x_load[i, j])
end
baseline_x = convert(Matrix{Int64}, baseline_x)

73×73 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  1  0  1  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  0
 0  1  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     

In [252]:
baseline_x
baseline_t = sum(baseline_x, dims=1)
print(size(baseline_t))

baseline_cost = cM*sum((1-baseline_t[P])*aE[P] for P=1:nP) + sum(baseline_x[i,j]*C[i,j] for i=1:nP, j=1:nP)

# print(sum((1-baseline_t[P])*aE[P] for P=1:nP))
# cost_term = sum(baseline_x[i,j]*C[i,j] for i=1:nP, j=1:nP)


(1, 73)

662.0

In [253]:
for i in 1:nP
    if aE[i] == 1 && baseline_t[i] == 0
        print("$(names[i])\n")
    end
end

In [254]:
for i in 1:nP
    if aE[i] == 1 && value.(t)[i] == 0
        print("$(names[i])\n$(i)")
    end
end

In [255]:
# Quality of match based on availability vs. baseline

ours = sum(x_sol[i,j]*qA[i,j] for i=1:nP, j=1:nP)
baseline = sum(baseline_x[i,j]*qA[i,j] for i=1:nP, j=1:nP)

print("Quality of match based on availability:\n")
print("Ours: $(ours)\n")
print("Baseline: $(baseline)\n")

Quality of match based on availability:
Ours: 3192.0
Baseline: 2673.0


In [256]:
# Quality of matche based on rank vs. baseline

ours = -sum(x_sol[i,j]*qR[i,j] for i=1:nP, j=1:nP)
baseline = -sum(baseline_x[i,j]*qR[i,j] for i=1:nP, j=1:nP)

print("Quality of match based on rank:\n")
print("Ours: $(ours)\n")
print("Baseline: $(baseline)\n")

Quality of match based on rank:
Ours: -383.0
Baseline: -400.0


In [257]:
# Quality of match based on interest vs. baseline

ours = sum(x_sol[i,j]*qI[i,j] for i=1:nP, j=1:nP)
baseline = sum(baseline_x[i,j]*qI[i,j] for i=1:nP, j=1:nP)

print("Quality of match based on interest:\n")
print("Ours: $(ours)\n")
print("Baseline: $(baseline)\n")

Quality of match based on interest:
Ours: 19.0
Baseline: 6.0
